In [ ]:
import holoviews as hv
import numpy as np
import panel as pn
from PIL import Image

hv.extension('bokeh')

In [ ]:
x = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png"))
imshape = x.shape
print(imshape)
img = hv.RGB(x, bounds=[0, 0, 320, -240])#.opts(invert_yaxis=True)
img

In [ ]:
#curve = hv.Curve(np.random.randn(50).cumsum())
a = imshape[1]/2 - imshape[1]/4
b = imshape[1]/2 + imshape[1]/4
c = -1*imshape[0]/2 - imshape[0]/4
d = -1*imshape[0]/2 + imshape[0]/4

curve = hv.Curve(([a,a,b,b], [d,c,c,d]))
curve

In [ ]:
curve_annotator = hv.annotate.instance()

In [ ]:
img*curve_annotator(curve, annotations={"Label":str})

In [ ]:
curve_annotator.annotated.dframe()

In [ ]:
import matplotlib.pyplot as plt
import torch
import kornia.geometry, kornia.augmentation

In [ ]:
im = np.array(Image.open("/home/joe/Documents/electricmayhem_tutorials/data/flower2.png")).astype(np.float32)/255
imt = torch.tensor(im).permute(2,0,1)[:3,:,:]
print(imt.shape)
plt.imshow(im);

In [ ]:
aug = kornia.augmentation.RandomAffine(scale=(0.5,1.5), translate=0.25, degrees=30)
im1 = aug(imt).detach()
im2 = aug(imt).detach()
plt.subplot(121)
plt.imshow(im1.squeeze(0).permute(1,2,0).numpy())
plt.subplot(122)
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());

In [ ]:
reg = kornia.geometry.ImageRegistrator(num_iterations=1000)

In [ ]:
%%time
model = reg.register(im1, im2)

In [ ]:
model.shape

In [ ]:
warped = kornia.geometry.homography_warp(im1, model, (240,320))

In [ ]:
plt.subplot(121)
plt.imshow(warped.squeeze(0).permute(1,2,0).detach().numpy())
plt.subplot(122)
plt.imshow(im2.squeeze(0).permute(1,2,0).numpy());